In [54]:
from __future__ import annotations

from pydantic import TypeAdapter
from functools import lru_cache

import requests

@lru_cache(maxsize=1)
def fetch_api_pokemons() -> dict[str, ApiPokemon]:
    ret = requests.get("https://pokemon-go-api.github.io/pokemon-go-api/api/pokedex.json")
    return ret.json()

In [92]:
from functools import lru_cache
from pydantic import BaseModel, Field, HttpUrl
from inflection import camelize

from typing import Any

class MyBaseModel(BaseModel):
    class Config:
        alias_generator = lambda x: camelize(x, uppercase_first_letter=False)


class ApiLangString(MyBaseModel):
    German: str = Field(alias="German")
    French: str = Field(alias="French")
    Italian: str = Field(alias="Italian")
    Japanese: str = Field(alias="Japanese")
    Korean: str = Field(alias="Korean")
    Spanish: str = Field(alias="Spanish")
    English: str = Field(alias="English")


class ApiAssets(MyBaseModel):
    image: str
    shiny_image: str


class ApiType(MyBaseModel):
    type: str
    names: ApiLangString


class ApiStats(MyBaseModel):
    stamina: int
    attack: int
    defense: int


class ApiEvolution(MyBaseModel):
    id: str
    form_id: str
    candies: int
    item: dict | None
    quests: list[dict]

class ApiForm(MyBaseModel):
    form: str
    costume: bool
    isFemale: bool
    image: HttpUrl
    shinyImage: HttpUrl


class ApiPokemon(MyBaseModel):
    id: str
    form_id: str
    dex_nr: int
    generation: int
    names: ApiLangString
    stats: ApiStats | None
    primary_type: ApiType
    secondary_type: ApiType | None
    pokemon_class: str | None
    assets: ApiAssets | None
    asset_forms: dict[str, ApiPokemon] | list[None]
    region_forms:  dict[str, ApiPokemon] | list[None]
    evolutions: list[ApiEvolution]


In [93]:
data = fetch_api_pokemons()

In [97]:
data[18]["assetForms"]

[{'form': 'ALOLA',
  'costume': None,
  'isFemale': False,
  'image': 'https://raw.githubusercontent.com/PokeMiners/pogo_assets/master/Images/Pokemon/Addressable%20Assets/pm19.fALOLA.icon.png',
  'shinyImage': 'https://raw.githubusercontent.com/PokeMiners/pogo_assets/master/Images/Pokemon/Addressable%20Assets/pm19.fALOLA.s.icon.png'},
 {'form': None,
  'costume': None,
  'isFemale': True,
  'image': 'https://raw.githubusercontent.com/PokeMiners/pogo_assets/master/Images/Pokemon/Addressable%20Assets/pm19.g2.icon.png',
  'shinyImage': 'https://raw.githubusercontent.com/PokeMiners/pogo_assets/master/Images/Pokemon/Addressable%20Assets/pm19.g2.s.icon.png'},
 {'form': None,
  'costume': None,
  'isFemale': False,
  'image': 'https://raw.githubusercontent.com/PokeMiners/pogo_assets/master/Images/Pokemon/Addressable%20Assets/pm19.icon.png',
  'shinyImage': 'https://raw.githubusercontent.com/PokeMiners/pogo_assets/master/Images/Pokemon/Addressable%20Assets/pm19.s.icon.png'},
 {'form': None,
  

In [95]:
v = TypeAdapter(ApiPokemon)

# for i, d in enumerate(data):
#     print(i)
p = v.validate_python(data[18])

ValidationError: 8 validation errors for ApiPokemon
assetForms.`dict[str,...]`
  Input should be a valid dictionary [type=dict_type, input_value=[{'form': 'ALOLA', 'costu...icon_019_01_shiny.png'}], input_type=list]
    For further information visit https://errors.pydantic.dev/2.5/v/dict_type
assetForms.list[none].0
  Input should be None [type=none_required, input_value={'form': 'ALOLA', 'costum...pm19.fALOLA.s.icon.png'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/none_required
assetForms.list[none].1
  Input should be None [type=none_required, input_value={'form': None, 'costume':...ets/pm19.g2.s.icon.png'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/none_required
assetForms.list[none].2
  Input should be None [type=none_required, input_value={'form': None, 'costume':...Assets/pm19.s.icon.png'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/none_required
assetFo

In [69]:
from pydantic import BaseModel
from inflection import camelize


class Voice(BaseModel):
    name: str
    language_code: str

    class Config:
        alias_generator = camelize

v = TypeAdapter(Voice)
v.validate_python({'Name': 'Filiz', 'LanguageCode': 'tr-TR'})

Voice(name='Filiz', language_code='tr-TR')